In [ ]:
# Student Management System
"""
Student Management System (SMS)

Features:
---------
1. User Authentication (with logging of activities).
2. CRUD Operations for Student Records:
   - Add
   - View
   - Search
   - Update
   - Delete
3. Report Generation (summary of students by grade).
4. Activity Logging (tracks all user actions).

Files Used:
-----------
- users.txt          : Stores usernames and passwords.
- students.txt       : Stores student records (roll, name, grade).
- activity_log.txt   : Stores user activities with timestamps.

Note:
-----
Default admin account is created automatically if no users exist:
    Username: admin
    Password: admin
"""
#importing modules required
import os
import getpass
import datetime
#store files path
#========================================
#file paths
#========================================
USER_FILE="users.txt"
STUDENT_FILE="students.txt"
LOG_FILE="activity_log.txt"
#user authentication and logging
#========================================
#logging functions
#========================================
def log_activity(username,action):
    """Log user actions with timestamp into the activity log file"""
    with open(LOG_FILE,"a") as log:
        log.write(f"{datetime.datetime.now()}-{username}-{action}\n")
#======================================
#user authentication functions
#======================================
def load_users():
    """Load users from file into a dictionary.Returns:dict{username:password}"""
    users={}
    if os.path.exists(USER_FILE):
        with open(USER_FILE,"r") as f:
            for line in f:
                if "," in line:
                    uname,pwd=line.strip().split(",",1)
                    users[uname]=pwd
    return users
#save user function
"""
    Save a new user to the user file.

    Args:
        username (str): The username.
        password (str): The password.
    """

def save_user(username,password):
    with open(USER_FILE,"a") as f:
        f.write(f"{username},{password}\n")

#authenticate user function(user authentication and logging)
def authenticate():
    """authenticate user login.returns username as a string if login successful,none otherwise"""
    users=load_users()
    username=input("enter username: ")
    password=getpass.getpass("enter password: ")

    if username in users and users[username]==password:
        log_activity(username,"Logged in")
        print("\n Login successful!\n")
        return username
    else:
        print("\n invalid credentials!\n")
        return None
#===========================================================    
#STUDENT DATA MANAGEMENT SYSTEM(CRUD OPERATIONS AND REPORTS)
#===========================================================
#add student function
def add_student(username):
    """add a new student"""
    roll=input("enter Roll Number: ")
    name=input("enter name: ")
    grade=input("enter grade: ")

    with open (STUDENT_FILE,"a") as f:
            f.write(f"{roll},{name},{grade}\n")

    log_activity(username,f"added student {roll}")
    print("student added successful!")

#view student function
def view_students():#display all student records
    if not os.path.exists(STUDENT_FILE):
        print("no records found")
        return
    with open(STUDENT_FILE,"r") as f:
        print("\n---student records---")
        for line in f:
            roll,name,grade=line.strip().split(",")
            print(f"Roll:{roll}|Name:{name}|grade:{grade}")

#search student function
def search_student():   #search student by roll number
    roll_no=input("enter roll number to search: ")
    found=False

    if os.path.exists(STUDENT_FILE):
        with open(STUDENT_FILE,"r") as f:
            for line in f:
                #skip empty or invalid lines
                parts=line.strip().split(",")
                if len(parts)!=3:
                    continue
                roll,name,grade=parts
                if roll==roll_no:
                    print(f"Found:Roll:{roll}|Name:{name}|grade:{grade}")
                    found=True
                    break

#update student function
def update_student(username):   #update a student by roll number
    roll_no=input("enter roll number to update: ")
    students=[]
    updated=False

    if os.path.exists(STUDENT_FILE):
        with open(STUDENT_FILE,"r") as f:
            students=f.readlines()

        with open(STUDENT_FILE,"w") as f:
            for line in students:
                roll,name,grade=line.strip().split(",")
                if roll==roll_no:
                    print(f"current data-Name:{name},grade:{grade}")
                    new_name=input("enter new name: ")
                    new_grade=input("enter new grade: ")
                    f.write(f"{roll},{new_name},{new_grade}\n")
                    updated=True
                    log_activity(username,f"updated student {roll}")
                else:
                    f.write(line)
        if updated:
            print("student record updated.")
        else:
            print("student not found.")

# student delete function
def delete_student(username):   #delete a student record by roll number
    roll_no=input("enter roll number to delete: ")
    students=[]
    deleted=False

    if os.path.exists(STUDENT_FILE):
        with open(STUDENT_FILE,"r")as f:
            students=f.readlines()
        with open(STUDENT_FILE,"w") as f:
            for line in students:
                #skip empty invalid lines
                parts=line.strip().split(",")
                if len(parts)!=3:
                    f.write(line)#keep invalid lines unchanged
                    continue
                roll,name,grade=parts
                if roll==roll_no:
                    deleted=True
                    #log deletion
                    log_activity(username,f"deleted student{roll}")
                else:
                    f.write(line)
    if deleted:
        print("student record deleted.")
    else:
        print("student not found.")

#report generation
def generate_report():      #generate a summary of student grades
    if not os.path.exists(STUDENT_FILE):
        print("no records available")
        return

    total=0
    grades={}
    with open(STUDENT_FILE,"r")as f:
        for line in f:
            roll,name,grade=line.strip().split(",")
            total+=1
            grades[grade]=grades.get(grade,0)+1
    print("\nstudent report")
    print(f"total students:{total}")
    for g,count in grades.items():
        print(f"grade{g}:{count}student(s)")
#===============================================
#MAIN PROGRAM(entry point menu and program flow and menu navigation)
#===============================================
def main():
    print("====student management system===")

    #ensure admin user exists
    if not os.path.exists(USER_FILE):
        print("no users found.create an admin account.")
        uname=input("set admin username: ")
        pwd=getpass.getpass("set admin password: ")
        save_user(uname,pwd)
        print("admin created!please restart the program.")
        return #exit after creating admin
    username=None
    while not username:
        username=authenticate()
    #menu
    while True:
        print("\n---main menu---")
        print("1.add student")
        print("2.view all students")
        print("3.search student")
        print("4.update student")
        print("5.delete student") 
        print("6.generate report")
        print("7.logout &exit")

        choice=input("enter choice: ")
        if choice=="1":
            add_student(username)
        elif choice=="2":
            view_students()
        elif choice=="3":
            search_student()
        elif choice=="4":
            update_student(username)
        elif choice=="5":
            delete_student(username)
        elif choice=="6":
            generate_report()
        elif choice=="7":
            log_activity(username,"logged out")
            print("Goodbye!")
            break
        else:
            print("invalid choice!please try again.")

if __name__ == "__main__":
    main()








      
        




            
